<a href="https://colab.research.google.com/github/bathaee/QGAN-EQ-GAN/blob/main/Noisy_SamplerQNN_final_QGAN_Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit --upgrade
!pip install qiskit-machine-learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 22.4 MB/s eta 0:00:00


In [ ]:
import torch
from qiskit import QuantumCircuit, execute, Aer,transpile
from qiskit.circuit.library import EfficientSU2,TwoLocal
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit.primitives import Sampler as RefSampler
from qiskit.primitives import Sampler,Estimator
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.visualization import plot_histogram
from qiskit import Aer
from qiskit_aer import AerSimulator
import qiskit_aer.noise as noise
from qiskit.utils import algorithm_globals
from qiskit.providers import Options
from qiskit.utils import QuantumInstance
import qiskit.quantum_info as qi
from qiskit_aer.noise import NoiseModel, amplitude_damping_error,pauli_error, depolarizing_error

In [57]:
#Custom Noise model for measurment, x and cx gates

# Error probabilities
prob_1 = 0.01  # 1-qubit gate
prob_2 = 0.01  # 2-qubit gate
prob_m = 0.01  # Measurment error

# Depolarizing quantum errors
error_1 = noise.depolarizing_error(prob_1, 1)
error_2 = noise.depolarizing_error(prob_2, 2)
error_meas = noise.pauli_error([('X',prob_m), ('I', 1 - prob_m)])

# Add errors to noise model
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['y'])
noise_model.add_all_qubit_quantum_error(error_2, ['cz'])
noise_model.add_all_qubit_quantum_error(error_meas, "measure")

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

#print(basis_gates)

In [58]:
shots = 10000
#Without Noise
sampler = Sampler(options={ "shots": shots, "seed": algorithm_globals.random_seed})
sam = Sampler(options={"backend": AerSampler(),"backend.set_options": AerSampler().set_options(noise_model=noise_model)})
#With Noise
sampler_n = AerSampler()
#sampler_n.set_options(noise_model=noise_model,shots=shots, seed=algorithm_globals.random_seed)
sampler_n.set_options(noise_model=noise_model)
#test
sampler_nn = RefSampler()
#sampler_n.set_options(noise_model=noise_model,shots=shots, seed=algorithm_globals.random_seed)
sampler_nn.set_options(noise_model=noise_model)

In [59]:
# Test if it works for SamplerQNN
qc_test = QuantumCircuit(2)
qc_test.h([0])
qc_test.cx([0],[1])
qnn = SamplerQNN(
        sampler = sampler_n,
        circuit= qc_test,
        input_params=[],
        weight_params=qc_test.parameters,
        sparse=False,
        ) 
qnn.forward(input_data=[], weights=[])

array([[0.48144531, 0.00976562, 0.01367188, 0.49511719]])

In [60]:
num_qubits = 2
qc = QuantumCircuit(num_qubits)
qc.h(qc.qubits)
qc_zofal = QuantumCircuit(num_qubits)
qc_zofal.h(qc.qubits)
qc_zofal_ = TwoLocal(num_qubits,
                     # Parameterized single qubit rotations
                     ['ry'],
                     'cz',  # Entangling gate
                     'full', # Entanglement structure: all to all
                     reps=1, # Number of layers
                     parameter_prefix='θ_g',
                     name='Generator',
                    insert_barriers=True)
qc_zofal.compose(qc_zofal_, inplace=True)
qc_zofal.decompose().draw()

┌─────────┐┌────────────┐ ░     ░ ┌────────────┐
q_0: ┤ U2(0,π) ├┤ Ry(θ_g[0]) ├─░──■──░─┤ Ry(θ_g[2]) ├
     ├─────────┤├────────────┤ ░  │  ░ ├────────────┤
q_1: ┤ U2(0,π) ├┤ Ry(θ_g[1]) ├─░──■──░─┤ Ry(θ_g[3]) ├
     └─────────┘└────────────┘ ░     ░ └────────────┘

In [61]:
qnn = SamplerQNN(
        sampler = sampler,
        circuit= qc_zofal,
        input_params=[],
        weight_params=qc_zofal.parameters,
        sparse=False,
    ) 
qnn.forward(input_data=[], weights=[0.1, 0.2, 0.3, 0.4])

array([[0.0487, 0.3713, 0.4349, 0.1451]])

In [62]:
#p = qc_zofal.assign_parameters(algorithm_globals.random.random(qc_zofal.num_parameters))
p = qc_zofal.assign_parameters([0.1,0.2,0.3,0.4])
print(p.decompose())

     ┌─────────┐┌─────────┐ ░     ░ ┌─────────┐
q_0: ┤ U2(0,π) ├┤ Ry(0.1) ├─░──■──░─┤ Ry(0.3) ├
     ├─────────┤├─────────┤ ░  │  ░ ├─────────┤
q_1: ┤ U2(0,π) ├┤ Ry(0.2) ├─░──■──░─┤ Ry(0.4) ├
     └─────────┘└─────────┘ ░     ░ └─────────┘


In [63]:
#weights_n = algorithm_globals.random.random(qc_zofal.num_parameters)
weights_n =[0.1,0.2,0.3,0.4]
p = qc_zofal.assign_parameters(weights_n)
qnn = SamplerQNN(
        sampler = sampler_n,
        circuit= p,
        input_params=[],
        weight_params=[],
        sparse=False,
    ) 
qnn.forward(input_data=[], weights=[])

array([[0.04980469, 0.35058594, 0.4609375 , 0.13867188]])

In [64]:
#Without Noise
def create_generator() -> TorchConnector:
    qnn = SamplerQNN(
        sampler = sampler,
        circuit= qc_zofal,
        input_params=[],
        weight_params=qc_zofal.parameters,
        sparse=False,
    )    
    initial_weights = algorithm_globals.random.random(qc_zofal.num_parameters)
    #print(qnn.weight_params)

    return TorchConnector(qnn, initial_weights)

generator = create_generator()
print(generator(torch.tensor([])).reshape(-1,1))
#print(generator.get_parameters())


tensor([[0.1152],
        [0.3278],
        [0.5354],
        [0.0216]], grad_fn=<ReshapeAliasBackward0>)


In [65]:
#With Noise
weights_n = algorithm_globals.random.random(qc_zofal.num_parameters)
def create_generator_n(weights_n) -> TorchConnector:
    p = qc_zofal.assign_parameters(weights_n)
    qnn = SamplerQNN(
        sampler = sampler_n,
        circuit= p,
        input_params=[],
        weight_params=p.parameters,
        sparse=False,
    )    
    return TorchConnector(qnn, [])

generator_n = create_generator_n(weights_n)
print(generator_n)
print(generator_n(torch.tensor([])).reshape(-1,1))   
print(generator.parameters) 

TorchConnector()
tensor([[0.0312],
        [0.3213],
        [0.5400],
        [0.1074]], grad_fn=<ReshapeAliasBackward0>)
<bound method Module.parameters of TorchConnector()>


In [66]:
#With Noise parametrized circuit
def create_generator_nn() -> TorchConnector:
    qnn = SamplerQNN(
        sampler = sampler_n,
        circuit= qc_zofal,
        input_params=[],
        weight_params=qc_zofal.parameters,
        sparse=False,
    )    
    initial_weights = algorithm_globals.random.random(qc_zofal.num_parameters)
    #print(qnn.weight_params)

    return TorchConnector(qnn, initial_weights)

generator_nn = create_generator_nn()
print(generator_nn(torch.tensor([])).reshape(-1,1))
#print(generator.get_parameters())

ERROR: Failed to load circuits: Invalid parameterized qobj: instruction param position out of range


QiskitMachineLearningError: ignored

backend = AerSimulator(noise_model=noise_model)
backend.set_options(noise_model=noise_model)
job = execute(p, backend)
result = job.result()
# Get the counts from the simulation result
counts = result.get_counts()
print(counts)

In [ ]:
from torch import nn


class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()

        self.linear_input = nn.Linear(input_size, 20)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.linear20 = nn.Linear(20, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        x = self.linear_input(input)
        x = self.leaky_relu(x)
        x = self.linear20(x)
        x = self.sigmoid(x)
        return x

In [ ]:
num_dim =1
discriminator = Discriminator(num_dim)
print(discriminator)

Discriminator(
  (linear_input): Linear(in_features=1, out_features=20, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.2)
  (linear20): Linear(in_features=20, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [ ]:
def adversarial_loss(input, target, w):
    bce_loss = target * torch.log(input) + (1 - target) * torch.log(1 - input)
    weighted_loss = w * bce_loss
    total_loss = -torch.sum(weighted_loss)
    return total_loss

In [ ]:
from torch.optim import Adam

lr = 0.01  # learning rate
b1 = 0.7  # first momentum parameter
b2 = 0.999  # second momentum parameter

generator_optimizer = Adam(generator.parameters(), lr=lr, betas=(b1, b2), weight_decay=0.005)
discriminator_optimizer = Adam(discriminator.parameters(), lr=lr, betas=(b1, b2), weight_decay=0.005)

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib import cm

def plot_training_progress():
    # we don't plot if we don't have enough data
    if len(generator_loss_values) < 2:
        return

    clear_output(wait=True)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))

    # Generator Loss
    ax1.set_title("Loss")
    ax1.plot(generator_loss_values, label="generator loss", color="royalblue")
    ax1.plot(discriminator_loss_values, label="discriminator loss", color="magenta")
    ax1.legend(loc="best")
    ax1.set_xlabel("Iteration")
    ax1.set_ylabel("Loss")
    ax1.grid()

    # Relative Entropy
    ax2.set_title("Relative entropy")
    ax2.plot(entropy_values)
    ax2.set_xlabel("Iteration")
    ax2.set_ylabel("Relative entropy")
    ax2.grid()

    plt.show()

In [ ]:
import time
from scipy.stats import multivariate_normal, entropy

grid_elements = [[1],[2],[3],[4]]
prob_data = [0.1, 0.3, 0.5, 0.1]
n_epochs = 10

num_qnn_outputs = 4

generator_loss_values = []
discriminator_loss_values = []
entropy_values = []

start = time.time()
for epoch in range(n_epochs):

    valid = torch.ones(num_qnn_outputs, 1, dtype=torch.float)
    fake = torch.zeros(num_qnn_outputs, 1, dtype=torch.float)

    # Configure input
    real_dist = torch.tensor(prob_data, dtype=torch.float).reshape(-1, 1)

    # Configure samples
    samples = torch.tensor(grid_elements, dtype=torch.float)
    disc_value = discriminator(samples)

    # Generate data
    gen_dist = generator_n(torch.tensor([])).reshape(-1, 1)
    

    # Train generator
    generator_optimizer.zero_grad()
    generator_loss = adversarial_loss(disc_value, valid, gen_dist)

    # store for plotting
    generator_loss_values.append(generator_loss.detach().item())

    generator_loss.backward(retain_graph=True)
    generator_optimizer.step()

    # Train Discriminator
    discriminator_optimizer.zero_grad()

    real_loss = adversarial_loss(disc_value, valid, real_dist)
    fake_loss = adversarial_loss(disc_value, fake, gen_dist.detach())
    discriminator_loss = (real_loss + fake_loss) / 2

    # Store for plotting
    discriminator_loss_values.append(discriminator_loss.detach().item())

    discriminator_loss.backward()
    discriminator_optimizer.step()

    ##
    qnode = generator.qnode
    weights = qnode.get_parameters()



    entropy_value = entropy(gen_dist.detach().squeeze().numpy(), prob_data)
    entropy_values.append(entropy_value)

    plot_training_progress()

elapsed = time.time() - start
print(f"Fit in {elapsed:0.2f} sec")

AttributeError: ignored

In [ ]:
with torch.no_grad():
    generated_probabilities = generator().numpy()

with torch.no_grad():
    generated_probabilities = generator().numpy()

In [ ]:
fig = plt.figure(figsize=(18, 9))
import numpy as np

# Generated CDF
gen_prob_grid = np.cumsum(generated_probabilities)

ax1 = fig.add_subplot(1, 3, 1)
ax1.set_title("Generated CDF")
ax1.set_ylim(-0.1, 1.1)
ax1.scatter(grid_elements,gen_prob_grid)

# Real CDF
real_prob_grid = np.cumsum(prob_data)

ax2 = fig.add_subplot(1, 3, 2)
ax2.set_title("True CDF")
ax2.set_ylim(-0.1, 1.1)
ax2.scatter(grid_elements,real_prob_grid)

# Difference
ax3 = fig.add_subplot(1, 3, 3)
ax3.set_title("Difference between CDFs")
ax3.set_ylim(-0.1, 0.2)
ax3.scatter(grid_elements,real_prob_grid-gen_prob_grid)
plt.show()

In [ ]:
grid_elements = [[1],[2],[3],[4]]
samples = torch.tensor(grid_elements, dtype=torch.float)
disc_value = discriminator(samples)
print(grid_elements)
print(samples)
print(disc_value)

[[1], [2], [3], [4]]
tensor([[1.],
        [2.],
        [3.],
        [4.]])
tensor([[0.5327],
        [0.5131],
        [0.5037],
        [0.4943]], grad_fn=<SigmoidBackward0>)


In [ ]:
gen_dist = generator(torch.tensor([])).reshape(-1, 1)
print(torch.tensor([]))
print(gen_dist.detach())

tensor([])
tensor([[0.1057],
        [0.4431],
        [0.2776],
        [0.1737]])


In [ ]:
prob_data = [0.1, 0.3, 0.5, 0.1]
real_dist = torch.tensor(prob_data, dtype=torch.float).reshape(-1, 1)
valid = torch.ones(4, 1, dtype=torch.float)
fake = torch.ones(4, 1, dtype=torch.float)
print(real_dist)
print(valid)

tensor([[0.1000],
        [0.3000],
        [0.5000],
        [0.1000]])
tensor([[1.],
        [1.],
        [1.],
        [1.]])


In [ ]:
generator_optimizer.zero_grad()


In [ ]:
generator_loss = adversarial_loss(disc_value, valid, gen_dist.detach())
print(generator_loss)

tensor(0.6749, grad_fn=<NegBackward0>)


In [ ]:
generator_loss.backward(retain_graph=True)
generator_optimizer.step()

In [ ]:
discriminator_optimizer.zero_grad()

real_loss = adversarial_loss(disc_value, valid, real_dist)
fake_loss = adversarial_loss(disc_value, fake, gen_dist.detach())
discriminator_loss = (real_loss + fake_loss) / 2
print(discriminator_loss)

tensor(0.6757, grad_fn=<DivBackward0>)


In [ ]:
discriminator_loss.backward()
discriminator_optimizer.step()

In [ ]:
from scipy.stats import multivariate_normal, entropy
entropy_value = entropy(gen_dist.detach().squeeze().numpy(), prob_data)
print(entropy_value)
print(gen_dist.detach().squeeze().numpy())
print(prob_data)

0.11109840318877957
[0.10568087 0.44305262 0.27760056 0.17366599]
[0.1, 0.3, 0.5, 0.1]


In [ ]:
disc_value = discriminator(samples)
print(disc_value)

tensor([[0.5602],
        [0.5575],
        [0.5638],
        [0.5707]], grad_fn=<SigmoidBackward0>)


In [ ]:
print(generator(torch.tensor([])).reshape(-1,1))

tensor([[0.1057],
        [0.4431],
        [0.2776],
        [0.1737]], grad_fn=<ReshapeAliasBackward0>)
